# Dynamic Investment Strategies with Market Regimes

## Overview

This notebook demonstrates a comprehensive Python-based solution for **Dynamic Investment Strategies with Market Regimes**. The system dynamically adjusts portfolio allocations based on predicted market regimes, improving upon traditional Modern Portfolio Theory (MPT) by addressing its sensitivity to static input assumptions.

### Key Features

- **Regime Identification**: Automatically identifies market regimes using economic indicators
- **Regime Forecasting**: Predicts future market regimes using machine learning
- **Dynamic Portfolio Optimization**: Adjusts allocations based on regime predictions
- **Multi-Horizon Analysis**: Tests strategy robustness across different time periods
- **Real Economic Data**: Integrates with FRED API for authentic economic indicators

### Architecture Overview

The system follows a 5-stage architecture:

1. **Historical Regime Identification** (Unsupervised Learning)
2. **Regime Forecasting** (Supervised Learning)
3. **Regime-Conditional Capital Market Assumptions**
4. **Portfolio Optimization** (Modern Portfolio Theory)
5. **Strategy Simulation** (Backtesting & Evaluation)

## Setup and Installation

First, let's import the necessary libraries and initialize our system:

In [ ]:
# Core imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Our regime strategy system
from src.regime_strategies.main import RegimeStrategyApp
from src.regime_strategies.backtesting.visualization import PerformanceVisualizer

print("✓ Libraries imported successfully")
print("Ready to demonstrate Dynamic Investment Strategies!")

## 1. Basic System Initialization

Let's start by initializing our regime strategy application:

In [ ]:
# Initialize the regime strategy system
app = RegimeStrategyApp()

print("System Configuration:")
print(f"- Regime Identification Method: {app.config.regimes.identification_method}")
print(f"- Default Optimization Method: {app.config.optimization.default_method}")
print(f"- Forecasting Models: {app.config.forecasting.ensemble_models}")
print(f"- FRED API Configured: {'Yes' if app.config.data.fred_api_key else 'No (using synthetic data)'}")

## 2. Regime-Aware Strategy Backtest

Now let's run a comprehensive backtest using our regime-aware strategy. This example demonstrates how the system:

1. Identifies historical market regimes using economic indicators
2. Trains machine learning models to forecast future regimes  
3. Dynamically optimizes portfolio weights based on regime predictions
4. Compares performance against static MPT benchmarks

In [ ]:
# Define our investment universe
assets = ['SPY', 'TLT', 'GLD']  # S&P 500, Treasury Bonds, Gold
start_date = '2022-01-01'
end_date = '2022-12-31'

print(f"Running regime-aware backtest...")
print(f"Assets: {assets}")
print(f"Period: {start_date} to {end_date}")
print()

# Run the backtest
result = app.run_backtest(
    start_date=start_date,
    end_date=end_date,
    assets=assets,
    strategy_name='demo_strategy',
    save_results=False
)

print("\n" + "="*60)
print("REGIME-AWARE STRATEGY RESULTS")
print("="*60)
print(f"Strategy Name: {result['strategy_name']}")
print(f"Backtest Period: {result['backtest_period']}")
print(f"Total Return: {result['total_return']:.2%}")
print(f"Annualized Return: {result['annualized_return']:.2%}")
print(f"Volatility: {result['volatility']:.2%}")
print(f"Sharpe Ratio: {result['sharpe_ratio']:.3f}")
print(f"Maximum Drawdown: {result['maximum_drawdown']:.2%}")
print(f"Regime Prediction Accuracy: {result['regime_prediction_accuracy']:.1%}")

print(f"\nBenchmark Comparison:")
print(f"Static MPT Outperformance: {result.get('static_mpt_outperformance', 0):.2%}")
print(f"Regime Return Advantage: {result.get('regime_advantage_return', 0):+.2%}")
print(f"Regime Sharpe Advantage: {result.get('regime_advantage_sharpe', 0):+.3f}")

## 3. Portfolio Weight Evolution Visualization

Let's visualize how the portfolio weights evolved over time based on regime predictions:

In [ ]:
# Get the backtest results for visualization
backtest_result = app.backtest_engine.results
visualizer = PerformanceVisualizer(app.config)

# Create weight evolution plot
if hasattr(backtest_result, 'portfolio_weights'):
    weights_df = backtest_result.portfolio_weights
    
    # Create interactive plot
    fig = go.Figure()
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    for i, asset in enumerate(weights_df.columns):
        fig.add_trace(go.Scatter(
            x=weights_df.index,
            y=weights_df[asset] * 100,  # Convert to percentage
            mode='lines',
            name=asset,
            line=dict(width=2, color=colors[i % len(colors)]),
            hovertemplate=f'{asset}: %{{y:.1f}}%<extra></extra>'
        ))
    
    fig.update_layout(
        title="Portfolio Weight Evolution Over Time",
        xaxis_title="Date",
        yaxis_title="Weight (%)",
        hovermode='x unified',
        height=500
    )
    
    fig.show()
    
    print("Portfolio allocation changes based on regime predictions:")
    print(weights_df.describe())
else:
    print("Portfolio weights data not available")

## 4. Regime Transition Analysis

Let's examine how market regimes were identified and predicted during our backtest period:

In [ ]:
# Analyze regime transitions
if hasattr(backtest_result, 'predicted_regimes') and hasattr(backtest_result, 'actual_regimes'):
    predicted_regimes = backtest_result.predicted_regimes
    actual_regimes = backtest_result.actual_regimes
    
    # Create regime transition plot
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=['Predicted vs Actual Regimes', 'Regime Distribution'],
        vertical_spacing=0.15
    )
    
    # Regime transitions plot
    fig.add_trace(
        go.Scatter(
            x=predicted_regimes.index,
            y=predicted_regimes,
            mode='markers+lines',
            name='Predicted Regimes',
            marker=dict(size=8, color='blue'),
            line=dict(width=2, color='blue')
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=actual_regimes.index,
            y=actual_regimes,
            mode='markers+lines',
            name='Actual Regimes',
            marker=dict(size=8, color='red', symbol='x'),
            line=dict(width=2, color='red', dash='dash')
        ),
        row=1, col=1
    )
    
    # Regime distribution histogram
    regime_counts = predicted_regimes.value_counts().sort_index()
    regime_labels = [f'Regime {int(r)}' for r in regime_counts.index]
    
    fig.add_trace(
        go.Bar(
            x=regime_labels,
            y=regime_counts.values,
            name='Regime Frequency',
            marker_color='lightblue'
        ),
        row=2, col=1
    )
    
    fig.update_layout(
        title="Market Regime Analysis",
        height=700
    )
    
    fig.update_yaxes(title_text="Regime", row=1, col=1)
    fig.update_yaxes(title_text="Frequency", row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    
    fig.show()
    
    # Calculate prediction accuracy
    accuracy = (predicted_regimes == actual_regimes).mean()
    print(f"\nRegime Prediction Accuracy: {accuracy:.1%}")
    
    # Show regime mapping
    regime_mapping = {
        0: "Slowing (Low Growth, Low Inflation)",
        1: "Heating (High Growth, High Inflation)", 
        2: "Growing (High Growth, Low Inflation)",
        3: "Stagflation (Low Growth, High Inflation)"
    }
    
    print("\nRegime Interpretation (Investment Clock):")
    for regime_id, description in regime_mapping.items():
        if regime_id in regime_counts.index:
            print(f"  Regime {regime_id}: {description}")
else:
    print("Regime data not available for visualization")

## 5. Strategy Comparison: Max Sharpe vs Risk Parity

Let's compare different optimization methods within our regime-aware framework:

In [ ]:
# Test different optimization methods
optimization_methods = ['max_sharpe', 'risk_parity']
comparison_results = {}

for method in optimization_methods:
    print(f"\nRunning {method.replace('_', ' ').title()} strategy...")
    
    # Update configuration
    app.config.optimization.default_method = method
    
    # Run backtest
    result = app.run_backtest(
        start_date='2022-01-01',
        end_date='2022-12-31',
        assets=['SPY', 'TLT', 'GLD'],
        strategy_name=f'{method}_comparison',
        save_results=False
    )
    
    comparison_results[method] = result

# Create comparison table
comparison_df = pd.DataFrame({
    'Max Sharpe': {
        'Total Return': f"{comparison_results['max_sharpe']['total_return']:.2%}",
        'Sharpe Ratio': f"{comparison_results['max_sharpe']['sharpe_ratio']:.3f}",
        'Max Drawdown': f"{comparison_results['max_sharpe']['maximum_drawdown']:.2%}",
        'Regime Advantage': f"{comparison_results['max_sharpe'].get('regime_advantage_return', 0):+.2%}"
    },
    'Risk Parity': {
        'Total Return': f"{comparison_results['risk_parity']['total_return']:.2%}",
        'Sharpe Ratio': f"{comparison_results['risk_parity']['sharpe_ratio']:.3f}",
        'Max Drawdown': f"{comparison_results['risk_parity']['maximum_drawdown']:.2%}",
        'Regime Advantage': f"{comparison_results['risk_parity'].get('regime_advantage_return', 0):+.2%}"
    }
})

print("\n" + "="*60)
print("OPTIMIZATION METHOD COMPARISON")
print("="*60)
print(comparison_df.to_string())

# Determine winner
max_sharpe_return = comparison_results['max_sharpe']['total_return']
risk_parity_return = comparison_results['risk_parity']['total_return']

if risk_parity_return > max_sharpe_return:
    winner = "Risk Parity"
    advantage = risk_parity_return - max_sharpe_return
else:
    winner = "Max Sharpe"
    advantage = max_sharpe_return - risk_parity_return

print(f"\n🏆 Winner: {winner} (+{advantage:.2%} return advantage)")

## 6. Multi-Horizon Analysis

Let's test the strategy's robustness across multiple time horizons using our multi-horizon analysis framework:

In [ ]:
# Import multi-horizon analyzer
import sys
import os
sys.path.append(os.getcwd())

try:
    from multi_horizon_analysis import MultiHorizonAnalyzer
    
    # Initialize analyzer
    analyzer = MultiHorizonAnalyzer(app)
    
    print("Running multi-horizon analysis (limited for demo)...")
    
    # Run analysis with limited scope for notebook demo
    results_df = analyzer.run_multi_horizon_analysis(
        overall_start='2022-01-01',
        overall_end='2023-06-30',
        max_combinations=6  # Limited for notebook
    )
    
    # Display results summary
    successful_results = results_df[results_df['success'] == True]
    
    if len(successful_results) > 0:
        print("\nMulti-Horizon Results Summary:")
        print(f"Successful Backtests: {len(successful_results)}/{len(results_df)}")
        print(f"Outperformance Rate: {successful_results['outperformed_static'].mean():.1%}")
        print(f"Average Return Advantage: {successful_results['regime_return_advantage'].mean():+.2%}")
        
        # Show detailed results
        display_cols = ['start_date', 'end_date', 'regime_return_advantage', 'outperformed_static']
        display_df = successful_results[display_cols].copy()
        display_df['regime_return_advantage'] = display_df['regime_return_advantage'].apply(lambda x: f"{x:+.2%}")
        display_df['outperformed_static'] = display_df['outperformed_static'].apply(lambda x: "✓" if x else "✗")
        display_df.columns = ['Start Date', 'End Date', 'Return Advantage', 'Outperformed']
        
        print("\nDetailed Results:")
        print(display_df.to_string(index=False))
        
except ImportError as e:
    print(f"Multi-horizon analyzer not available: {e}")
    print("You can run the multi_horizon_analysis.py script separately for full analysis")

## 7. FRED API Integration for Real Economic Data

Our system can integrate with the Federal Reserve Economic Data (FRED) API for real economic indicators. Let's check the setup status:

In [ ]:
import os

# Check FRED API setup
def check_fred_setup():
    try:
        from fredapi import Fred
        fredapi_available = True
    except ImportError:
        fredapi_available = False
    
    fred_api_key = os.getenv('FRED_API_KEY')
    
    print("FRED API Setup Status:")
    print("=" * 30)
    print(f"fredapi library: {'✓ Available' if fredapi_available else '✗ Missing (pip install fredapi)'}")
    print(f"FRED API Key: {'✓ Configured' if fred_api_key else '✗ Not configured'}")
    
    if fred_api_key and fredapi_available:
        try:
            fred = Fred(api_key=fred_api_key)
            test_data = fred.get_series('GDP', limit=1)
            print(f"API Connection: ✓ Working")
            return True
        except Exception as e:
            print(f"API Connection: ✗ Failed ({e})")
            return False
    
    return False

fred_ready = check_fred_setup()

if not fred_ready:
    print("\n📋 FRED API Setup Instructions:")
    print("1. Get free API key: https://fred.stlouisfed.org/")
    print("2. Install library: pip install fredapi")
    print("3. Set environment variable: FRED_API_KEY=your_key_here")
    print("\n⚠️  Currently using synthetic data for regime identification")
else:
    print("\n✅ FRED API ready - using real economic data!")

## 8. Interactive Dashboard Creation

Let's create a comprehensive interactive dashboard showing all key metrics:

In [ ]:
# Create comprehensive dashboard
def create_strategy_dashboard(regime_result, static_result=None):
    """Create comprehensive strategy dashboard"""
    
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            'Cumulative Returns', 'Portfolio Weights Over Time',
            'Regime Transitions', 'Performance Metrics',
            'Risk Analysis', 'Return Distribution'
        ],
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"secondary_y": False}]
        ],
        vertical_spacing=0.12,
        horizontal_spacing=0.1
    )
    
    # Get backtest results
    backtest_result = app.backtest_engine.results
    
    if hasattr(backtest_result, 'portfolio_returns'):
        returns = backtest_result.portfolio_returns
        cum_returns = (1 + returns).cumprod()
        
        # 1. Cumulative Returns
        fig.add_trace(
            go.Scatter(
                x=cum_returns.index,
                y=cum_returns.values,
                mode='lines',
                name='Regime Strategy',
                line=dict(color='blue', width=2)
            ),
            row=1, col=1
        )
        
        # 2. Portfolio Weights
        if hasattr(backtest_result, 'portfolio_weights'):
            weights = backtest_result.portfolio_weights
            colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
            
            for i, asset in enumerate(weights.columns):
                fig.add_trace(
                    go.Scatter(
                        x=weights.index,
                        y=weights[asset],
                        mode='lines',
                        stackgroup='weights',
                        name=f'{asset} Weight',
                        line=dict(color=colors[i % len(colors)])
                    ),
                    row=1, col=2
                )
        
        # 3. Regime Transitions
        if hasattr(backtest_result, 'predicted_regimes'):
            regimes = backtest_result.predicted_regimes
            
            fig.add_trace(
                go.Scatter(
                    x=regimes.index,
                    y=regimes,
                    mode='markers+lines',
                    name='Predicted Regimes',
                    marker=dict(size=8, color='red')
                ),
                row=2, col=1
            )
        
        # 4. Performance Metrics (Bar Chart)
        metrics = {
            'Total Return': regime_result['total_return'] * 100,
            'Sharpe Ratio': regime_result['sharpe_ratio'],
            'Max Drawdown': regime_result['maximum_drawdown'] * 100
        }
        
        fig.add_trace(
            go.Bar(
                x=list(metrics.keys()),
                y=list(metrics.values()),
                name='Performance Metrics',
                marker_color=['green', 'blue', 'red']
            ),
            row=2, col=2
        )
        
        # 5. Risk Analysis - Drawdown
        drawdown = (cum_returns / cum_returns.expanding().max()) - 1
        
        fig.add_trace(
            go.Scatter(
                x=drawdown.index,
                y=drawdown * 100,
                mode='lines',
                name='Drawdown',
                fill='tonexty',
                line=dict(color='red')
            ),
            row=3, col=1
        )
        
        # 6. Return Distribution
        fig.add_trace(
            go.Histogram(
                x=returns * 100,  # Convert to percentage
                nbinsx=30,
                name='Daily Returns',
                marker_color='lightblue'
            ),
            row=3, col=2
        )
    
    # Update layout
    fig.update_layout(
        title="Dynamic Investment Strategy Dashboard",
        height=1000,
        showlegend=True
    )
    
    # Update axes labels
    fig.update_yaxes(title_text="Cumulative Return", row=1, col=1)
    fig.update_yaxes(title_text="Weight", row=1, col=2)
    fig.update_yaxes(title_text="Regime", row=2, col=1)
    fig.update_yaxes(title_text="Value", row=2, col=2)
    fig.update_yaxes(title_text="Drawdown (%)", row=3, col=1)
    fig.update_yaxes(title_text="Frequency", row=3, col=2)
    fig.update_xaxes(title_text="Returns (%)", row=3, col=2)
    
    return fig

# Create and display dashboard
print("Creating interactive dashboard...")
dashboard = create_strategy_dashboard(result)
dashboard.show()
print("✓ Dashboard created successfully!")

## 9. Advanced Example: Custom Regime Definition

Let's demonstrate how to customize the regime identification logic for specific market conditions:

In [ ]:
# Example: Custom regime analysis
def analyze_regime_characteristics(app_instance):
    """Analyze the characteristics of identified regimes"""
    
    # Load economic data
    try:
        econ_data = app_instance.data_loader.load_economic_indicators(
            start_date=pd.to_datetime('2020-01-01').date(),
            end_date=pd.to_datetime('2023-12-31').date()
        )
        
        print("Economic Indicators Analysis:")
        print("=" * 40)
        print(f"Data Shape: {econ_data.shape}")
        print(f"Date Range: {econ_data.index[0]} to {econ_data.index[-1]}")
        print(f"Available Indicators: {list(econ_data.columns)}")
        
        # Show regime distribution
        if 'CPI_signal' in econ_data.columns and 'CLI_signal' in econ_data.columns:
            # Create regime labels based on Investment Clock
            regimes = []
            for idx, row in econ_data.iterrows():
                if pd.notna(row['CPI_signal']) and pd.notna(row['CLI_signal']):
                    if row['CLI_signal'] > 0 and row['CPI_signal'] > 0:
                        regimes.append('Heating')
                    elif row['CLI_signal'] > 0 and row['CPI_signal'] <= 0:
                        regimes.append('Growing')
                    elif row['CLI_signal'] <= 0 and row['CPI_signal'] > 0:
                        regimes.append('Stagflation')
                    else:
                        regimes.append('Slowing')
                else:
                    regimes.append('Unknown')
            
            regime_counts = pd.Series(regimes).value_counts()
            print("\nRegime Distribution:")
            for regime, count in regime_counts.items():
                print(f"  {regime}: {count} periods ({count/len(regimes)*100:.1f}%)")
                
            # Visualize regime evolution
            fig = go.Figure()
            
            regime_numeric = pd.Series(regimes).map({
                'Slowing': 0, 'Growing': 1, 'Heating': 2, 'Stagflation': 3, 'Unknown': -1
            })
            
            fig.add_trace(go.Scatter(
                x=econ_data.index[:len(regime_numeric)],
                y=regime_numeric,
                mode='markers+lines',
                name='Market Regimes',
                marker=dict(size=8, color=regime_numeric, colorscale='Viridis')
            ))
            
            fig.update_layout(
                title="Market Regime Evolution (Investment Clock)",
                yaxis_title="Regime",
                xaxis_title="Date",
                yaxis=dict(
                    tickmode='array',
                    tickvals=[0, 1, 2, 3],
                    ticktext=['Slowing', 'Growing', 'Heating', 'Stagflation']
                )
            )
            
            fig.show()
            
    except Exception as e:
        print(f"Error loading economic data: {e}")
        print("Using synthetic data - consider setting up FRED API for real data")

# Run regime analysis
print("Analyzing regime characteristics...")
analyze_regime_characteristics(app)

## 10. Export Results and Next Steps

Let's save our analysis results and provide guidance for further exploration:

In [ ]:
# Create summary of all results
summary_data = {
    'Analysis': ['Basic Strategy', 'Max Sharpe', 'Risk Parity'],
    'Total Return': [
        f"{result['total_return']:.2%}",
        f"{comparison_results['max_sharpe']['total_return']:.2%}",
        f"{comparison_results['risk_parity']['total_return']:.2%}"
    ],
    'Sharpe Ratio': [
        f"{result['sharpe_ratio']:.3f}",
        f"{comparison_results['max_sharpe']['sharpe_ratio']:.3f}",
        f"{comparison_results['risk_parity']['sharpe_ratio']:.3f}"
    ],
    'Max Drawdown': [
        f"{result['maximum_drawdown']:.2%}",
        f"{comparison_results['max_sharpe']['maximum_drawdown']:.2%}",
        f"{comparison_results['risk_parity']['maximum_drawdown']:.2%}"
    ]
}

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*80)
print("COMPREHENSIVE ANALYSIS SUMMARY")
print("="*80)
print(summary_df.to_string(index=False))

# Save results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"regime_strategy_analysis_{timestamp}.csv"
summary_df.to_csv(filename, index=False)
print(f"\n💾 Results saved to: {filename}")

print("\n" + "="*80)
print("NEXT STEPS & RECOMMENDATIONS")
print("="*80)
print("""
🔬 FURTHER ANALYSIS:
• Run multi_horizon_analysis.py for comprehensive robustness testing
• Experiment with fred_integration_example.py for real economic data
• Try different asset combinations and time periods
• Test alternative regime identification methods

📈 OPTIMIZATION OPPORTUNITIES:
• Implement transaction costs and rebalancing constraints  
• Add risk budgeting and portfolio constraints
• Explore alternative regime forecasting models (XGBoost, Neural Networks)
• Test sector-specific regime strategies

🔧 TECHNICAL ENHANCEMENTS:
• Set up FRED API for real economic data (see fred_integration_example.py)
• Implement real-time regime monitoring
• Add performance attribution analysis
• Create automated reporting dashboard

📊 VISUALIZATION OPTIONS:
• Run visualization_demo.py for advanced interactive charts
• Create custom regime transition analysis
• Build rolling performance attribution plots
• Generate regime-specific risk decomposition charts

⚠️  IMPORTANT CONSIDERATIONS:
• Current analysis uses synthetic economic data for demonstration
• Consider transaction costs in real-world implementation  
• Validate regime identification with domain expertise
• Test extensively before live deployment
""")

print("\n✅ Analysis complete! Thank you for exploring Dynamic Investment Strategies.")

## Conclusion

This notebook has demonstrated the complete **Dynamic Investment Strategies with Market Regimes** system, showcasing:

### ✅ Key Achievements

1. **Regime-Aware Portfolio Optimization**: Dynamic allocation based on predicted market regimes
2. **Machine Learning Integration**: Ensemble models for regime forecasting
3. **Multiple Optimization Methods**: Max Sharpe and Risk Parity strategies
4. **Comprehensive Backtesting**: Full simulation with proper data handling
5. **Interactive Visualizations**: Rich charts for analysis and interpretation
6. **Multi-Horizon Analysis**: Robustness testing across different time periods
7. **Real Data Integration**: FRED API support for authentic economic indicators

### 🎯 Core Value Proposition

The system addresses key limitations of traditional Modern Portfolio Theory by:
- **Dynamic Adaptation**: Portfolios adjust to changing market conditions
- **Regime Awareness**: Explicit modeling of different economic environments  
- **Forecasting Integration**: Predictive models guide allocation decisions
- **Comprehensive Testing**: Rigorous validation across multiple scenarios

### 🔬 Methodology Highlights

- **Investment Clock Framework**: Economic theory-based regime identification
- **Ensemble Machine Learning**: Robust regime forecasting with multiple models
- **Point-in-Time Data**: Proper handling to avoid look-ahead bias
- **Benchmark Comparisons**: Quantified outperformance vs static strategies

### 📈 Performance Insights

The analysis reveals the conditions where regime-aware strategies excel and provides tools to:
- Identify optimal regime forecasting accuracy thresholds
- Compare different optimization approaches
- Understand regime transition impacts on performance
- Evaluate strategy robustness across market cycles

---

**Ready to implement regime-aware investment strategies? Start with the example files and gradually customize for your specific use case!**